# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
text = browser.html

In [ ]:
# Extract all rows of data
html = soup(text, 'html.parser')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
# Loop through the scraped data to create a list of rows
headers = [h.text for h in html.findAll('th')]
data = [ [ d.text for d in row.findAll('td')] for row in html.findAll('tr', class_='data-row')]

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.DataFrame(data, columns=headers)

In [ ]:
# Confirm DataFrame was created successfully
df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
df.dtypes

In [ ]:
# Change data types for data analysis
# You can use the Pandas astype and to_datetime methods to accomplish this task.

# id is already obj, does not need to change
df['terrestrial_date'] = pd.to_datetime(df['terrestrial_date'])

df['sol'] = df['sol'].astype(int)
df['ls'] = df['ls'].astype(int)
df['month'] = df['month'].astype(int)

df['min_temp'] = df['min_temp'].astype(float)
df['pressure'] = df['pressure'].astype(float)

In [ ]:
# Confirm type changes were successful by examining data types again
#change int32 to int64?

df.dtypes

In [ ]:
df.info()

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
mars_months = df.groupby('month').size().sort_index(ascending=True)
mars_months

In [ ]:
# 2. How many Martian days' (sols) worth of data are there?
total_sols = df['sol'].nunique()
total_sols

In [ ]:
# 3. What is the average low temperature by month?
avg_low_temp = df.groupby('month')['min_temp'].mean()
avg_low_temp

In [ ]:
# Plot the average temperature by month
plt.figure(figsize=(10, 6))
plt.bar(avg_low_temp.index, avg_low_temp.values, color='steelblue')
plt.title('Average Temperature by Month')
plt.xlabel('month')
plt.ylabel('Temperature in Celsius')
plt.xticks(range(1, 13))  
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
# coldest month
coldest_month = avg_low_temp.idxmin()
coldest_temp = avg_low_temp.min()

# hottest month
hottest_month = avg_low_temp.idxmax()
hottest_temp = avg_low_temp.max()

print(coldest_month, coldest_temp)
print(hottest_month, hottest_temp)


In [ ]:
# Identify the coldest and hottest months in Curiosity's location
sort_low_temp = avg_low_temp.sort_values()


plt.figure(figsize=(10, 6))
plt.bar(sort_low_temp.index, sort_low_temp.values, color='steelblue', label='Average Low Temperature')
plt.bar(coldest_month, coldest_temp, color='steelblue')  
plt.bar(hottest_month, hottest_temp, color='steelblue')  
plt.title('Average Low Temperature by Month (Coldest to Hottest)')
plt.xlabel('month')
plt.ylabel('Temperature in Celsius')
plt.xticks(range(1, 13), sort_low_temp.index)  

plt.show()

In [ ]:
# 4. Average pressure by Martian month
avg_pressure = df.groupby('month')['pressure'].mean()
avg_pressure



In [ ]:
# Plot the average pressure by month
plt.figure(figsize=(10, 6))
plt.bar(avg_pressure.index, avg_pressure.values, color='steelblue')
plt.title('Average Pressure by Month')
plt.xlabel('month')
plt.ylabel('Atmospheric Pressure')
plt.xticks(range(1, 13), avg_pressure.index)  

plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
# Calculating #  of terrestrial days in a Martian year
terr_days = (df[df['month'] == 12]['terrestrial_date'].max() - df[df['month'] == 1]['terrestrial_date'].min()).days

# df needs min temp
terr_df = df[['terrestrial_date', 'min_temp']]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(terr_df.index, terr_df['min_temp'])
plt.xlabel('Number of terrestrial days')
plt.ylabel('Minimum Temperature')

plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
# terrestrial (earth) days are there in a Martian year
terr_df.to_csv('terrestrial_days_martian_year.csv', index=False)

In [ ]:
browser.quit()